In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
from PIL import Image, ImageOps
import numpy as np
from scipy import ndimage
import cv2
import os
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras.preprocessing import image
import matplotlib.pyplot as plt 
import tensorflow
from tensorflow.keras.layers import Input, BatchNormalization, ReLU, ELU, \
     Conv2D, Dense, MaxPool2D, AvgPool2D, GlobalAvgPool2D, Concatenate
from tensorflow.keras import optimizers
import splitfolders

Spliting the dataset into training and testing

In [21]:
input_dir = './Ekush_Cleaned'
output_dir = './Ekush_Cleaned_Output'
splitfolders.ratio(input_dir, output_dir, seed=1337, ratio=(0.8, 0.2))

Copying files: 63115 files [00:05, 11235.53 files/s]


In [25]:
# Change root_dir as you need
root_dir = './Ekush_Cleaned_Output'

for root, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        file_path = os.path.join(root, filename)

        # Load the image
        img = cv2.imread(file_path)
        
        # Changing the image color space from BGR to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Invert the image
        img = cv2.bitwise_not(img)

        # Resize the image
        img = cv2.resize(img, (32, 32))

        # Save the image
        cv2.imwrite(file_path, img)


In [26]:
train_data_dir = './Ekush_Cleaned_Output/train'
validation_data_dir = './Ekush_Cleaned_Output/val'

#listing = os.listdir(train_data_dir)

In [27]:
img_width, img_height, c = 32, 32, 3
batch_size=128

In [28]:
datagen = ImageDataGenerator(rescale=1. / 255)

#automatically retrive images and their classes for training and validation
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width,img_height),
        batch_size=batch_size,
        shuffle= False,
        class_mode = 'categorical')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width,img_height),
        batch_size=batch_size,
        shuffle= False,
        class_mode = 'categorical')

Found 50476 images belonging to 50 classes.
Found 12639 images belonging to 50 classes.


In [29]:
IMG_SHAPE = 32, 32, 3
#classes=3

In [30]:
nb_epoch = 30
nb_train_samples = 34439
nb_validation_samples = 8520

In [31]:
def bn_rl_conv(x, filters, kernel_size):
    #x = BatchNormalization()(x)
    x = ELU()(x)
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               padding='same')(x)
    x =  tensorflow.keras.layers.Dropout(0.1)(x)
    
    return x

In [32]:
def dense_block(tensor, k, reps):
    for _ in range(reps):
        x = bn_rl_conv(tensor, filters=4*k, kernel_size=1)
        x = bn_rl_conv(x, filters=k, kernel_size=3)
        #x =  tensorflow.keras.layers.Dropout(0.1)(x)
        tensor = Concatenate()([tensor, x])
    return tensor

In [33]:
def transition_layer(x, theta):
    f = int(tensorflow.keras.backend.int_shape(x)[-1] * theta)
    x = bn_rl_conv(x, filters=f, kernel_size=1)
    #x =  tensorflow.keras.layers.Dropout(0.1)(x)
    #x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
    return x

In [34]:
IMG_SHAPE = 32, 32, 3
k = 8
theta = 0.5
repetitions = 6, 6, 8


input = Input(IMG_SHAPE)

In [50]:
x = Conv2D(2*k, 7, strides=2, padding='same')(input)
x =  tensorflow.keras.layers.Dropout(0.1)(x)
x = MaxPool2D(3, strides=2, padding='same')(x)
#x = BatchNormalization()(x)

In [51]:
for reps in repetitions:
    d = dense_block(x, k, reps)
    x = transition_layer(d, theta)
    #x = BatchNormalization()(x)
    
#x = BatchNormalization()(x)
x = GlobalAvgPool2D()(d)

output = Dense(50, activation='softmax')(x)

from tensorflow.keras import Model 
model = Model(input, output)

In [52]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 16, 16, 16)   2368        input_1[0][0]                    
__________________________________________________________________________________________________
dropout_45 (Dropout)            (None, 16, 16, 16)   0           conv2d_53[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 8, 8, 16)     0           dropout_45[0][0]                 
____________________________________________________________________________________________

In [53]:

optimizer = optimizers.Adam()
opt = tensorflow.keras.optimizers.RMSprop()

In [54]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [55]:
his1= model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=30,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size)

Epoch 1/30
269/269 [==============================] - 14s 35ms/step - loss: 4.0028 - accuracy: 0.0128 - val_loss: 3.9100 - val_accuracy: 0.0372
Epoch 2/30
269/269 [==============================] - 8s 30ms/step - loss: 3.9671 - accuracy: 0.0216 - val_loss: 4.0727 - val_accuracy: 0.0356
Epoch 3/30
269/269 [==============================] - 8s 30ms/step - loss: 3.9270 - accuracy: 0.0215 - val_loss: 3.9071 - val_accuracy: 0.0237
Epoch 4/30
269/269 [==============================] - 8s 30ms/step - loss: 3.7857 - accuracy: 0.0604 - val_loss: 3.5400 - val_accuracy: 0.0923
Epoch 5/30
269/269 [==============================] - 8s 30ms/step - loss: 3.6008 - accuracy: 0.0838 - val_loss: 3.0970 - val_accuracy: 0.2057
Epoch 6/30
269/269 [==============================] - 8s 30ms/step - loss: 3.3945 - accuracy: 0.1347 - val_loss: 3.2235 - val_accuracy: 0.1580
Epoch 7/30
269/269 [==============================] - 8s 30ms/step - loss: 3.2065 - accuracy: 0.1712 - val_loss: 3.0727 - val_accuracy: 0.161